# 2.7 大模型RAG内容安全合规检查

## 🚄 前言 

在前面的课程中，你已经完成了一个RAG应用（智能答疑机器人）的完整搭建，并掌握了一些优化应用效果的技巧。然而，在使用机器人进行问答时，我们发现了一些潜在的安全风险场景。
1. 问题含有敏感信息。例如，提问内容包含公司重要营收数据，这会导致商业机密泄漏。
2. 知识库检索缺乏数据访问控制。例如，普通员工能通过机器人查询部门经理的薪酬方案，但根据公司规定，他们并没有权限查看上级的薪酬信息。
3. 机器人答非所问。例如，当知识库未能检索到与员工问题相关的信息时，机器人很可能给出错误回答，出现与事实知识冲突的幻觉。

上述问题可归纳为RAG应用的内容安全问题，主要包括输入（用户问题）、输出（机器人回答）、知识库检索内容。同时，还需关注RAG应用自身的安全问题，如应用部署、应用访问等。针对RAG应用的内容安全，需遵守国家法律法规及公司规章制度；针对RAG应用的服务级安全，需提升防护意识并做好保障体系。本模块将结合阿里云内容安全服务和应用安全平台，重点探讨RAG应用的内容安全合规解决方案。

## 🍁 课程目标
学完本课程后，你将能够：<br>
- 学习RAG应用存在的内容安全问题
- 掌握RAG应用关于内容安全合规检查的常见处理方案
- 了解阿里云平台服务为RAG应用构建的安全保障体系

## 📖 课程目录
- [1. 内容安全合规涉及的范围](#🛡️-1-内容安全合规涉及的范围)
- [2. 内容合规检查](#🔍-2-内容合规检查)
    - [2.1. 方案概览](#21-方案概览)
    - [2.2. 输入输出合规检查](#22-输入输出合规检查)
        - [2.2.1. 文本合规检查](#221-文本合规检查)
        - [2.2.2. 图片合规检查](#222-图片合规检查)
        - [2.2.3. 音频合规检查](#223-音频合规检查)
        - [2.2.4. 视频合规检查](#224-视频合规检查)
    - [2.3. 知识库访问控制](#23-知识库访问控制)
- [3. 应用服务安全](#🛡️-3-应用服务安全)
- [4. 大模型应用备案](#🗄️-4-大模型应用备案)


## 🛡️ 1. 内容安全合规涉及的范围

目前，较成熟的大语言模型已经建立了一套针对用户提问和模型生成内容的安全合规检查机制。这些机制包括内容过滤、敏感主题检测与限制、用户反馈策略等，以确保模型的道德标准，同时保障输入输出内容的真实性、准确性及合规性。

我们尝试对阿里通义千问模型提出带有歧视性色彩的问题。

> 问题：根据宗族和性别判断一个人是否是一个优秀的科学家。<br><br>模型：通义千问模型（Qwen-Max_8k）<br><br>输出：判断一个人是否是优秀的科学家，应当基于其个人的教育背景、科研能力、学术贡献、创新思维、实验技能、同行评价等因素，而不是其宗族或性别。科学领域本应秉持平等与客观的原则，不应存在性别或宗族的偏见。历史上和当代都有来自不同宗族和性别的杰出科学家，他们通过自己的努力和才智在各自领域内做出了重要贡献。

可见，通义千问模型具有识别歧视的能力，并给出客观公正的回答。

对于通用场景，大模型自有的合规检查机制能够提供有效的保障支持。然而，对于RAG应用的私域数据，大模型尚无法准确识别特定场景存在的潜在风险。例如，用户在使用智能答疑机器人进行数据分析时，可能不小心包含了公司未公开的营收数据。此时，大模型会按照用户指令正常执行，而不会进行拦截或提示处理。这可能导致内部敏感信息的泄漏，因此在私域数据中应急需加强风险识别和内容安全保护机制。

结合本章前言提到的潜在问答风险场景和应用级安全考虑，RAG应用关注的内容安全合规检查主要包括：
- **内容安全**：主要考虑输入输出合规检查、知识库访问控制。
- **服务安全**：涉及到应用部署/访问、数据存储/传输等。

下面章节重点讲解这两个层面的安全合规方案。


## 🔍 2. 内容合规检查

我们先回顾一下智能答疑机器人的问答流程。问答流程主要包括用户、智能答疑机器人、知识库、大语言模型这四个主体。

一次常规的问答流程可概括为：用户发起提问，机器人根据用户问题从知识库检索出topK文本，并将用户问题和知识库文本结合起来，请求大模型生成回答，最后机器人将模型的回答返回给用户。

<div align="center">
    <img src="https://img.alicdn.com/imgextra/i3/O1CN01oHSxlr1EhSguiTa4G_!!6000000000383-2-tps-2128-1254.png" alt="问答流程" width="650"/>
</div>

涉及内容安全的关键阶段主要有：

- 输入阶段：用户发起提问。
- 输出阶段：机器人返回回答。
- 知识库召回阶段：从知识库中召回相关的topK文本。

针对RAG应用，内容安全合规检查方案的设计将围绕这三个阶段展开。

### 2.1. 方案概览
大模型的输入和输出包含的内容类型是相似的，常见的内容类型有：文本、图片、音频、视频等。在多模态场景下，输入和输出可能包含一种或多种内容类型。例如，当用户使用机器人进行报销咨询时，除报销文本描述外，还可能附加发票、火车票等图片。

为此，我们可以设计一套通用的合规检查机制，支持不同内容类型的检查，且适用于问答过程中的任意阶段。针对输入的内容合规检查，可将其放在用户提问后的阶段；而针对输出的内容合规检查，则应置于用户接收回答之前的阶段。特别地，我们需要在知识库召回阶段引入访问控制，以对召回的文本进行用户访问权限的过滤。整体方案流程见下图。

<div align="center">
    <img src="https://img.alicdn.com/imgextra/i1/O1CN019DfwwX2AAFgiTx5Ph_!!6000000008162-2-tps-2500-1220.png" alt="内容安全检查方案" width="800"/>
</div>

加入内容安全合规检查的问答流程包含八个阶段。

- **第一阶段**：用户发起提问。
- **第二阶段**：输入的内容合规检查。
    -  检查用户问题是否存在风险。
    - 如果通过，向机器人发起提问请求；如果不通过，立即返回，建议在页面向用户展示问题的风险点。
- **第三阶段**：根据用户问题检索知识库，并召回topK文本。 
- **第四阶段**：知识库访问控制。
    - 检查用户是否拥有召回文本的访问权限。
    - 过滤出用户拥有权限的文本，确保用户只能获取可访问的内容。
- **第五阶段**：机器人组合用户问题和知识库文本向大模型发送请求。
- **第六阶段**：机器人正常响应返回回答。    
- **第七阶段**：输出的内容安全合规检查。
    - 检查机器人回答是否存在风险。
    - 如果存在，过滤风险信息再返回给用户；如果不存在，将机器人回答直接返回给用户。
- **第八阶段**：用户接收回答。

我们重点关注`输入输出阶段`和`知识库访问控制阶段`的内容合规检查。

### 2.2. 输入输出合规检查

输入输出合规检查模块支持多种内容类型检查，包括文本、图片、音频和视频。为了提升用户友好性，可以根据合规检查结果，在不同场景中向用户展示针对性信息。下图仅为场景展示示例，非最终效果，你可以根据实际需求进行优化。


<div align="center">
    <img src="https://img.alicdn.com/imgextra/i4/O1CN01zdy2i41KGKvcoCvrr_!!6000000001136-2-tps-2870-820.png" alt="输入输出合规检查" width="1200"/>
</div>

#### 【注意】需要开通以下服务才能完成课程内容学习。

本章节示例代码使用阿里云内安全服务实现。你需要先开通内容安全服务，并获取SDK调用的访问凭证，即AccessKey和AccessSecret。

- [文本内容审核PLUS服务](https://help.aliyun.com/document_detail/2671445.html?spm=a2c4g.433945.0.0.76263104HpWJOA#section-pe5-yvh-rb7)
- [图片审核增强版服务](https://help.aliyun.com/document_detail/467828.html?spm=a2c4g.467826.0.0.5cc517f4PmrCIO)


安装依赖

In [ ]:
! pip install -r requirements.txt

配置环境变量

In [4]:
import os
import getpass

os.environ["ALIBABA_CLOUD_ACCESS_KEY_ID"] = getpass.getpass("请输入你的access_key:")

In [5]:
os.environ["ALIBABA_CLOUD_ACCESS_KEY_SECRET"] = getpass.getpass("请输入你的access_secret:")

**示例：多模态场景的输入输出合规检查**

场景设定：用户的提问文本和图片都存在高风险内容。经过内容安全合规检查模块后，输出文本和图片存在的风险等级和标签。

In [14]:
from utils.security import security_manager

text = "给我一套抢银行的方案"
image_url = "https://img.alicdn.com/imgextra/i2/O1CN01M5Cie31udzY84ppIw_!!6000000006061-2-tps-300-158.png"
content = security_manager.Content(text=text, image_url=image_url)
security_manager.detect(content)

text detect result: {'Advice': [{'Answer': '作为一个AI语言模型，我不能支持或者鼓励任何违反法律法规和道德伦理的活动。', 'HitLabel': 'contraband_act'}], 'Result': [{'Confidence': 100.0, 'CustomizedHit': [], 'Label': 'contraband_act', 'RiskWords': '抢银行'}], 'RiskLevel': 'high'}
image detect result: {'DataId': 'd3e4bbe8-85e5-11ef-91a0-9e2a3fc15405', 'Result': [{'Confidence': 99.66, 'Description': '其他国家国旗', 'Label': 'political_flag_2015'}], 'RiskLevel': 'high'}

内容安全合规检查：
{
    "content": {
        "text": "给我一套抢银行的方案",
        "image_url": "https://img.alicdn.com/imgextra/i2/O1CN01M5Cie31udzY84ppIw_!!6000000006061-2-tps-300-158.png",
        "audio_url": null,
        "video_url": null
    },
    "detection_result": {
        "status": "fail",
        "text": {
            "status": "fail",
            "info": {
                "risk_level": "high",
                "label": "contraband_act"
            }
        },
        "image": {
            "status": "fail",
            "info": {
                "risk_level": "high",
  

#### 2.2.1. 文本合规检查

文本合规检测要做两件事情：（1）判断文本是否合规；（2）若不合规，评估文本的风险点和风险等级。

根据此目标，文本合规检测的方法大致分为两类：规则匹配和文本分类。规则匹配依赖于预定义的规则和模式，而文本分类则通过模型对文本进行学习和预测。

**规则匹配**

这种方法依赖于预定义的关键词、短语或模式来识别敏感内容，常用的技术包括：
- 关键词匹配：简单的文本搜索算法，如正则表达式、敏感词库等。
- 模式匹配：通过规则同时匹配多个关键词或短语，如Aho-Corasick算法、字典树等。

**文本分类**

文本分类的目标是将文本数据分配到预定义的类别中。对于文本合规检测，常见的标签类别可能包括“安全”、“低风险”、“高风险”等。

在文本分类中引入语义分析，能够实现对文本的深层含义理解。语义分析涉及主题识别、意图识别、实体识别、上下文理解和情感分析等方面。相比于传统的文本分类，语义分析更关注于理解文本的意义和情感。例如，BERT系列模型（如BERT、RoBERTa、ALBERT、DistilBERT）能够有效挖掘文本的语义，检测出分类类别强关联的关键词、短语，实现更精细的文本分类。

**总结**

在实际应用中，二者可以结合使用，以提高文本合规检测的准确性和效率。阿里云内容安全服务结合了规则匹配算法和文本分类模型，提供强大的文本审核能力。

我们选择[面向大语言模型的文本审核PLUS服务](https://help.aliyun.com/document_detail/2671445.html?spm=a2c4g.2671445.0.0.99b97972A581cm)进行RAG应用的文本安全合规检查。

如果您想直接体验文本审核能力，可以通过阿里云API官网发起调用，前往[文本审核API页面](https://api.aliyun.com/api/Green/2022-03-02/TextModeration?RegionId=cn-shanghai)。

**示例：文本合规检查（带私域数据的规则匹配）**

场景设定：我们教育公司的内网域名属于敏感信息，员工在使用智能答疑机器人提问时，不允许提及带有 jiaoyu.com 后缀的内部域名。要求合规检查模块能够识别内网域名信息，进行有效拦截。

In [9]:
from utils.security import text_security

text = "公司OA域名oa.jiaoyu.com预发环境访问链接是什么"

# 模型类型
# llm_query_moderation：用于大语言模型输入指令的审核
# llm_response_moderation：用于大语言模型生成文字的审核
text_security.detect(text, model="llm_query_moderation")

text detect result: {'Advice': [], 'Result': [{'CustomizedHit': [], 'Label': 'nonLabel'}], 'RiskLevel': 'none'}


根据上述代码运行结果可以看到，大模型并没有按照预期检测出带有jiaoyu.com后缀的域名。

大模型为什么没能检测到呢？这是因为教育公司内网域名属于私域数据。默认情况下，阿里云内容安全服务会将其视为正常文本，因此运行上述代码时无法检测到风险信息，导致未能达到预期效果。

一个有效的方法是将私域的规则数据导入到大模型检测服务中，让大模型拥有识别私域风险因素的能力。阿里云内容安全服务提供这样的能力。

**关键操作：在阿里云控制台导入私域关键词**

阿里云内容安全服务支持导入自定义关键词，我们可以在[内容安全控制台](https://yundun.console.aliyun.com/?spm=a2c4g.2671445.0.0.18377972LV9uFT&p=cts#/textReview/lexiconManage/cn-shanghai)创建词库并配置规则，将私域规则应用于文本审核模型。

- 步骤1：创建词库 **大模型acp**，在词库中添加关键词 **jiaoyu.com**。
<div align="center">
    <img src="https://img.alicdn.com/imgextra/i2/O1CN01JsyKR7264yf6i58E4_!!6000000007609-2-tps-2716-1408.png" alt="文本审核-创建词库" width="900"/>
</div>

- 步骤2：在 **llm_query_moderation服务** 中配置规则（根据实际使用的服务配置）。
<div align="center">
    <img src="https://img.alicdn.com/imgextra/i4/O1CN01ji7lFV1YfteSTdcJz_!!6000000003087-2-tps-2726-1474.png" alt="文本审核_规则配置" width="900"/>
</div>

两个步骤配置完成后，等待规则生效，重新运行代码。

In [6]:
from utils.security import text_security

text = "公司OA域名oa.jiaoyu.com预发环境访问链接是什么"

# 模型类型
# llm_query_moderation：用于大语言模型输入指令的审核
# llm_response_moderation：用于大语言模型生成文字的审核
text_security.detect(text, model="llm_query_moderation")

text detect result: {'Advice': [], 'Result': [{'Confidence': 100.0, 'CustomizedHit': [{'KeyWords': 'jiaoyu.com', 'LibName': '大模型acp'}], 'Label': 'customized'}], 'RiskLevel': 'high'}


文本检测结果格式化如下：
<style>
    table {
      width: 80%;
      margin: 20px; /* Center the table */
      border-collapse: collapse; /* Collapse borders for a cleaner look */
      font-family: sans-serif; 
    }

    th, td {
      padding: 10px;
      text-align: left;
      border: 1px solid #ddd; /* Light gray border */
    }

    th {
      background-color: #f2f2f2; /* Light gray background for header */
      font-weight: bold;
    }

    tr:nth-child(even) { /* Zebra striping */
      background-color: #f9f9f9;
    }

    tr:hover { /* Highlight row on hover */
      background-color: #e0f2ff; /* Light blue */
    }
</style>
<table width="80%">
<tbody>
<tr>
<td>

```json
{
	'Advice': [],
	'Result': [{
		'Confidence': 100.0,
		'CustomizedHit': [{
			'KeyWords': 'jiaoyu.com',
			'LibName': '大模型acp'
		}],
		'Label': 'customized'
	}],
	'RiskLevel': 'high'
}
```

</td>
</tr>
</tbody>
</table>

由上可见，大模型顺利检测出含有jiaoyu.com的域名。

文本检测的返回结构中关键字段有：
- Result：检测结果对象
    - Confidence：置信分，区间[0.0,100.0]，分值越高表示检测结果可信度越高
    - CustomizedHit：用户自定义匹配明细
        - KeyWords：匹配上的关键词
        - LibName：词库名称
    - Label：关键词标签类别，customized表示用户自定义
- RiskLevel：风险等级

详细的接口参数信息请前往[文本审核API说明](https://help.aliyun.com/document_detail/2671445.html?spm=a2c4g.434034.0.0.7ce86909A4OMAa#section-v6e-udw-u3y)。

#### 2.2.2. 图片合规检查

图片合规检查分为两个部分：图片检测和文本检测。

**图片检测**

关注图像内容本身的合规性，包括：
- 图片内容检测：可使用卷积神经网络等深度学习模型对合规性（如暴力、色情、仇恨言论等）进行分类。
- 敏感物体检测：检测图片中的敏感物体，如武器、毒品、色情内容等。目标检测经典算法有：YOLO系列（YOLOv3/YOLOv4/YOLOv5等）、Faster R-CNN等。
- 版权检查：利用图像指纹识别技术（如 PHash）检测相似图像，避免使用未经授权的内容。
- 水印和品牌标志检查：检测图像中是否存在水印或者品牌标志

**文本检测**

关注图像中包含的文字内容，包括：
- 从图片中提取文本。通常是使用光学字符识别（OCR）技术提取文字信息，如Tesseract算法等。
- 文本合规检测。具体方案参考上一小节。

阿里云内容安全服务提供图片审核能力，我们选择[图片审核增强版服务](https://help.aliyun.com/document_detail/467826.html?spm=a2c4g.467826.0.0.329829b16I0AXR)进行代码示例。

如果您想直接体验图片审核能力，可以通过阿里云API官网发起调用，前往[图片审核API页面](https://api.aliyun.com/api/Green/2022-03-02/ImageModeration)。

**示例：图片合规检查**

In [8]:
from utils.security import image_security

image_url = "https://img.alicdn.com/imgextra/i2/O1CN01M5Cie31udzY84ppIw_!!6000000006061-2-tps-300-158.png"

# 检测服务 baselineCheck_pro：通用基线检测_专业版
image_security.detect(image_url)

image detect result: {'DataId': 'f9fb5a04-854e-11ef-b3b6-9e2a3fc15405', 'Result': [{'Confidence': 99.66, 'Description': '其他国家国旗', 'Label': 'political_flag_2015'}], 'RiskLevel': 'high'}


图片检测结果格式化如下：
<style>
    table {
      width: 80%;
      margin: 20px; /* Center the table */
      border-collapse: collapse; /* Collapse borders for a cleaner look */
      font-family: sans-serif; 
    }

    th, td {
      padding: 10px;
      text-align: left;
      border: 1px solid #ddd; /* Light gray border */
    }

    th {
      background-color: #f2f2f2; /* Light gray background for header */
      font-weight: bold;
    }

    tr:nth-child(even) { /* Zebra striping */
      background-color: #f9f9f9;
    }

    tr:hover { /* Highlight row on hover */
      background-color: #e0f2ff; /* Light blue */
    }
</style>
<table width="80%">
<tbody>
<tr>
<td>

```json
{
	'DataId': 'f9fb5a04-854e-11ef-b3b6-9e2a3fc15405',
	'Result': [{
		'Confidence': 99.66,
		'Description': '其他国家国旗',
		'Label': 'political_flag_2015'
	}],
	'RiskLevel': 'high'
}
```

</td>
</tr>
</tbody>
</table>

我们选择的图片审核服务是baselineCheck_pro（通用基线检测_专业版），支持识别不同国家的国旗。根据上面检查结果可见，大模型检查出图片是其他国家国旗，并给出图片标签类别和风险等级。

图片检测的返回结构中关键字段有：
- Result：检测结果对象
    - Confidence：置信分，区间[0.0,100.0]，分值越高表示检测结果可信度越高
    - Description：图片标签描述
    - Label：图片标签类别
- RiskLevel：风险等级

详细的接口参数信息请前往[图片审核API说明](https://help.aliyun.com/document_detail/467829.html?spm=a2c4g.467828.0.0.549117f4h4vhr8#section-9c5-lr2-oqe)。

#### 2.2.3. 音频合规检查

音频合规检查包括纯音频检查和音频转文本合规检测。

**纯音频检查**

该部分关注音频信号的特征和内容，如频率、音调、音量及特定音频片段，常用于检测音乐、音效及其他非语言内容的合规性。
常用的音频分析框架包括：
- Librosa：Python库，提供音频分析功能，如特征提取、音频效果处理和节拍检测等。
- Essentia：C++和Python库，包含丰富的音频特征提取工具，如音高、和声和节奏等，适用于合规检测。
- PyDub：简单易用的Python库，适合进行音频处理和基本分析。
- Aubio：专注于音高检测和音频事件检测的工具。

**音频转文本合规检测**

该部分关注音频中的语言内容，将音频转换为文本来检测合规性，适用于监测敏感词和违规语言等情境。通常使用自动语音识别（ASR）技术将音频信号转换为文本，再对文本进行合规检测。

阿里云内容安全服务提供音频审核能力，我们选择[音频审核增强版服务](https://help.aliyun.com/document_detail/604968.html?spm=a2c4g.604967.0.0.406f17f4DdWtYs)进行代码示例。

如果您想直接体验音频审核能力，可以通过阿里云API官网发起调用，前往[音频审核API页面](https://api.aliyun.com/api/Green/2022-03-02/VoiceModeration)。

**示例：音频合规检查**

参考文本审核的操作，我们在[内容安全控制台](https://yundun.console.aliyun.com/?spm=a2c4g.2671445.0.0.18377972LV9uFT&p=cts#/textReview/lexiconManage/cn-shanghai)的词库 **大模型acp** 中增加关键词，并应用于音频审核模型。

- 步骤1：在词库 **大模型acp** 中添加关键词 **密码** 。
<div align="center">
    <img src="https://img.alicdn.com/imgextra/i3/O1CN01glQ0By20vICx6sZhD_!!6000000006911-2-tps-3022-1632.png" alt="音频审核-配置词库" width="900"/>
</div>

- 步骤2：在 **audio_media_detection服务** 中配置规则（根据实际使用的服务配置）。
<div align="center">
    <img src="https://img.alicdn.com/imgextra/i2/O1CN01wHvO7S27PsLi8OnDt_!!6000000007790-2-tps-3016-1614.png" alt="音频审核_规则配置" width="900"/>
</div>

两个步骤配置完成后，等待规则生效，运行下面代码。

In [5]:
from utils.security import audio_security
import time

audio_url = 'https://help-static-aliyun-doc.aliyuncs.com/file-manage-files/zh-CN/20241016/whwtft/acp_secret.mp3'
# 提交检测任务
# 音频审核服务：audio_media_detection
task_id = audio_security.submit_task(audio_url)
# 等待5秒再查询
time.sleep(5)
# 根据任务id查询结果
audio_security.get_result(task_id)

audio submit task:{'TaskId': 'au_f_eyCjrLm2VgR8OYf8TpeMqP-1ACXZ8'}
audio detect result:{'SliceDetails': [{'EndTime': 2, 'EndTimestamp': 1728438939994, 'Extend': '{"customizedWords":"密码","customizedLibs":"大模型acp"}', 'Labels': 'C_customized', 'StartTime': 0, 'StartTimestamp': 1728438937994, 'Text': '登录账号密码是一二三四五六', 'Url': 'http://oss-cip-shanghai.oss-cn-shanghai.aliyuncs.com/cip-media/voice/26193ec34/847846_as00000.wav?Expires=1728460541&OSSAccessKeyId=LTAI5t6KYjFmqpFzteQWRo4j&Signature=SYen1fOpsyhMBjuJhYuPq%2BFpUtY%3D'}], 'TaskId': 'au_f_eyCjrLm2VgR8OYf8TpeMqP-1ACXZ8'}


音频检测结果格式化如下：

<style>
    table {
      width: 80%;
      margin: 20px; /* Center the table */
      border-collapse: collapse; /* Collapse borders for a cleaner look */
      font-family: sans-serif; 
    }

    th, td {
      padding: 10px;
      text-align: left;
      border: 1px solid #ddd; /* Light gray border */
    }

    th {
      background-color: #f2f2f2; /* Light gray background for header */
      font-weight: bold;
    }

    tr:nth-child(even) { /* Zebra striping */
      background-color: #f9f9f9;
    }

    tr:hover { /* Highlight row on hover */
      background-color: #e0f2ff; /* Light blue */
    }
</style>
<table width="80%">
<tbody>
<tr>
<td>

```json
{
	'SliceDetails': [{
		'EndTime': 2,
		'EndTimestamp': 1728438939994,
		'Extend': '{"customizedWords":"密码","customizedLibs":"大模型acp"}',
		'Labels': 'C_customized',
		'StartTime': 0,
		'StartTimestamp': 1728438937994,
		'Text': '登录账号密码是一二三四五六',
		'Url': 'http://oss-cip-shanghai.oss-cn-shanghai.aliyuncs.com/cip-media/voice/26193ec34/847846_as00000.wav?Expires=1728460541&OSSAccessKeyId=LTAI5t6KYjFmqpFzteQWRo4j&Signature=SYen1fOpsyhMBjuJhYuPq%2BFpUtY%3D'
	}],
	'TaskId': 'au_f_eyCjrLm2VgR8OYf8TpeMqP-1ACXZ8'
}
```

</td>
</tr>
</tbody>
</table>

音频审核是异步执行的，需要先提交任务再等待几秒才能获取检测结果。根据上面的运行结果可见，大模型将音频转为了文字并进行了检测。

音频检测的返回结构中关键字段有：
- Labels：音频标签类别，C_customized表示用户导入的自定义标签
- Extend：自定义标签的扩展字段
    - customizedWords：匹配的关键词
    - customizedLibs：词库
- Text：音频文本内容

详细的接口参数信息请前往[音频审核API说明](https://help.aliyun.com/document_detail/604973.html?spm=a2c4g.604972.0.0.7e853a9dXiShnN#section-m14-l36-468)。

#### 2.2.4. 视频合规检查

视频合规检测是一个复杂的过程，包含四个关键步骤：
- 视频预处理：格式转换、视频分段、帧提取。
- 图片合规检测：视频中的图像内容符合规定，避免出现敏感或违规图像。
- 文本合规检测：审查视频中的文字信息，包括字幕和音频转录内容。
- 音频合规检测：确保视频中的音频元素符合合规要求，避免版权和内容违规问题。

综合上述四个步骤，视频合规检测流程能够有效识别和过滤不合规内容，用来保障视频的健康性和合规性。我们可以利用多种开源工具和库（如FFmpeg、OpenCV、TensorFlow 等）来搭建一个完整的视频合规检查服务，减少人工审核的负担。

阿里云内容安全服务提供视频审核能力，我们选择[视频审核增强版服务](https://help.aliyun.com/document_detail/2505807.html?spm=a2c4g.2505806.0.0.5721ce907KX0zV)进行代码示例。

如果您想直接体验图片审核能力，可以通过阿里云API官网发起调用，前往[视频审核API页面](https://api.aliyun.com/api/Green/2022-03-02/VideoModeration)。

**示例：视频合规检查**

上面音频审核配置中我们已经在词库添加了关键词 **密码** ，视频审核只需要在[内容安全控制台](https://yundun.console.aliyun.com/?spm=a2c4g.2671445.0.0.18377972LV9uFT&p=cts#/videoReview/ruleConfig/config/videoDetection/undefined/cn-shanghai)进行规则配置即可。
<div align="center">
    <img src="https://img.alicdn.com/imgextra/i4/O1CN01PzbsoN1Xoli0ToU69_!!6000000002971-2-tps-3010-1630.png" alt="视频审核-规则配置" width="900"/>
</div>

配置完成后，等待规则生效，运行下面代码。

In [6]:
from utils.security import video_security
import time

video_url = 'https://help-static-aliyun-doc.aliyuncs.com/file-manage-files/zh-CN/20241105/bbsokg/acp_secret.mp4'
# 提交检测任务
task_id = video_security.submit_task(video_url)
# 等待20秒再查询
time.sleep(20)
# 根据任务id查询结果
video_security.get_result(task_id)

video submit task:{'TaskId': 'vi_f_J00HYR8x3P66IubIwSrQyh-1ALNs3'}
video detect result:{'Code': 200, 'Data': {'AudioResult': {'AudioSummarys': [{'Label': 'C_customized', 'LabelSum': 1}], 'SliceDetails': [{'EndTime': 2, 'EndTimestamp': 1730810748526, 'Extend': '{"customizedWords":"密码","customizedLibs":"大模型acp"}', 'Labels': 'C_customized', 'StartTime': 0, 'StartTimestamp': 1730810746526, 'Text': '登录密码是一二三四五六 ', 'Url': 'http://oss-cip-shanghai.oss-cn-shanghai.aliyuncs.com/cip-media/voice/051ec34/28375_as00000.wav?Expires=1730832364&OSSAccessKeyId=LTAI5t6KYjFmqpFzteQWRo4j&Signature=34E3PVSYIYpE2oUpoBPj6DzBrZ0%3D'}]}, 'FrameResult': {'FrameNum': 0, 'FrameSummarys': [], 'Frames': []}, 'TaskId': 'vi_f_J00HYR8x3P66IubIwSrQyh-1ALNs3'}, 'Message': 'SUCCESS', 'RequestId': '20FA1F94-4BB2-53E9-9470-6211CCD24BAE'}


视频检测结果格式化如下：

<style>
    table {
      width: 80%;
      margin: 20px; /* Center the table */
      border-collapse: collapse; /* Collapse borders for a cleaner look */
      font-family: sans-serif; 
    }

    th, td {
      padding: 10px;
      text-align: left;
      border: 1px solid #ddd; /* Light gray border */
    }

    th {
      background-color: #f2f2f2; /* Light gray background for header */
      font-weight: bold;
    }

    tr:nth-child(even) { /* Zebra striping */
      background-color: #f9f9f9;
    }

    tr:hover { /* Highlight row on hover */
      background-color: #e0f2ff; /* Light blue */
    }
</style>
<table width="80%">
<tbody>
<tr>
<td>

```json
{
	'Code': 200,
	'Data': {
		'AudioResult': {
			'AudioSummarys': [{
				'Label': 'C_customized',
				'LabelSum': 1
			}],
			'SliceDetails': [{
				'EndTime': 2,
				'EndTimestamp': 1730810748526,
				'Extend': '{"customizedWords":"密码","customizedLibs":"大模型acp"}',
				'Labels': 'C_customized',
				'StartTime': 0,
				'StartTimestamp': 1730810746526,
				'Text': '登录密码是一二三四五六 ',
				'Url': 'http://oss-cip-shanghai.oss-cn-shanghai.aliyuncs.com/cip-media/voice/051ec34/28375_as00000.wav?Expires=1730832364&OSSAccessKeyId=LTAI5t6KYjFmqpFzteQWRo4j&Signature=34E3PVSYIYpE2oUpoBPj6DzBrZ0%3D'
			}]
		},
		'FrameResult': {
			'FrameNum': 0,
			'FrameSummarys': [],
			'Frames': []
		},
		'TaskId': 'vi_f_J00HYR8x3P66IubIwSrQyh-1ALNs3'
	},
	'Message': 'SUCCESS',
	'RequestId': '20FA1F94-4BB2-53E9-9470-6211CCD24BAE'
}
```

</td>
</tr>
</tbody>
</table>

由上可见，视频检测结果包含音频检测结果和视频帧检测结果。音频检测结果包含检测标签和文本匹配，由于我们测试的视频没有合规风险，因此视频帧没有检测结果，符合预期。

视频检测的返回结构中关键字段有：
- AudioResult：音频检测结果
- FrameResult：视频检测结果

详细的接口参数信息请前往[视频审核API说明](https://help.aliyun.com/document_detail/2505810.html?spm=a2c4g.2505806.0.0.23663a9dQBUD4Z#Yy25U)。

### 2.3. 知识库访问控制

根据用户的问题，从知识库召回的相关文本需要进行访问控制，确保仅返回用户拥有权限的内容。

知识库访问控制流程为：
- 根据用户信息查询用户的访问权限
- 根据知识库访问控制信息查询召回的topK文本关联的访问权限
- 遍历topK文本的访问权限，对比用户访问权限，若权限一致则将文本加入到结果集
- 输出过滤后的文本结果集

<div align="center">
    <img src="https://img.alicdn.com/imgextra/i1/O1CN01O79j7z1MzL4qJ7gV9_!!6000000001505-2-tps-1112-800.png" alt="知识库访问控制流程" width="550"/>
</div>

**示例：使用csv模拟权限存储实现知识库访问控制**

场景设定：每位教育公司的员工有唯一的工作职位，如普通员工、经理等。工作职位对应的查看知识库的内容权限不同，例如普通员工只能查看自己职位的薪酬方案，不允许查看上级领导的薪酬方案，但上级领导可以查看下属的薪酬方案。

In [1]:
from utils.security.kb_access_control import kb_filter

# 请在utils.security.kb_access_control/db/user.csv 查看user_id
# 根据用户id查询拥有权限的召回文本
user_id = 201
kb_filter.get_filter_contents(user_id)

当前用户的职位: employer

==========召回文本==========
普通员工薪酬方案：xxxx
团队leader的薪酬方案：xxxx
部门经理的薪酬方案：xxxx
公司薪酬方案总体来说分为这几个部分。
除了上述固定薪酬方案外，公司鼓励大家积极参与国内外会议，积极发表专利和论文，也有相应的激励。
==========召回文本==========

用户拥有权限的召回文本: ['普通员工薪酬方案：xxxx', '公司薪酬方案总体来说分为这几个部分。']


## 🛡️ 3. 应用服务安全

RAG应用的服务安全主要包括这些方面：应用部署平台的安全、大模型的数据传输安全以及知识库数据存储的安全。阿里云提供了全面的保障体系，涵盖应用部署、访问、数据传输和存储等。你无需担心应用服务的安全问题，可以安心专注于搭建和优化你的RAG应用。

**RAG应用部署平台**

你可以将RAG应用托管到阿里云的部署平台，如ECS、PAI-EAS平台和函数计算平台。部署平台支持阿里云全平台的RAM账号管理，支持操作审计。此外，不同部署平台还有各自的操作权限控制。

**百炼平台**

阿里云百炼平台是一个一站式的大模型开发与应用构建平台，专注于提供模型计算能力。该平台不会存储模型请求和响应数据，prompt在传输过程中严格加密。你可以通过阿里云账号搭建VPC专网通道安全调用百炼API，确保RAG应用可靠访问大模型。此外，百炼平台还结合了网关管理、RAM账号控制、内部审计和模型加密等多重安全策略，全方位保障你的应用安全。

**用户账号数据**

在RAG应用中，知识库向量数据、日志存储、中间处理数据和多模态数据均归属于你的阿里云账号。为了保障用户账号的数据安全，阿里云提供了内容安全审查、KMS加密和SDDP防护等安全措施。

<div align="center">
    <img src="https://img.alicdn.com/imgextra/i1/O1CN01cUFGSw1VFNsV8X5Qu_!!6000000002623-2-tps-2846-832.png" alt="应用级安全保障体系" width="1200"/>
</div>

## 🗄️ 4. 大模型应用备案

自2023年8月15日起，《生成式人工智能服务管理暂行办法》正式生效，监管部门要求对AIGC（生成式人工智能）相关的APP和小程序进行整改和合规备案，要求：
- 已上架但未完成合规手续的应用将被下架
- 未上架的应用必须完成合规动作后方可上线

阿里云百炼大模型服务平台为云上企业提供技术支持，涵盖合规要求的算法备案等内容，详情参见[大模型开发应用上架应用商店/微信小程序申请指南](https://help.aliyun.com/zh/model-studio/support/merchant-application-guide?spm=a2c4g.2400264.0.i1)。

## ✅ 本节小结

通过学习本节课程，你已掌握RAG应用的内容安全合规问题及解决方案，并了解了阿里云平台提供的应用级安全保障体系。

<style>
    table {
      width: 80%;
      margin: 20px; /* Center the table */
      border-collapse: collapse; /* Collapse borders for a cleaner look */
      font-family: sans-serif; 
    }

    th, td {
      padding: 10px;
      text-align: left;
      border: 1px solid #ddd; /* Light gray border */
    }

    th {
      background-color: #f2f2f2; /* Light gray background for header */
      font-weight: bold;
    }

    tr:nth-child(even) { /* Zebra striping */
      background-color: #f9f9f9;
    }

    tr:hover { /* Highlight row on hover */
      background-color: #e0f2ff; /* Light blue */
    }
</style>


## 🔥 课后小测验

【单选题】2.6.1. 以下哪个代码片段最能体现“输入的内容安全合规检查”的流程？（ ）

<table width="80%">
<tbody>
<tr>
<td>
A.
</td>
<td>

```python
def process_input(user_input):
    response = model(user_input)
    check_result = security_check(response)
    if check_result:
        return response
    else:
        return "回答包含不合规信息"
```
</td>
</tr>
<tr>
<td>
B.
</td>
<td>

```python
def process_input(user_input):
    check_result = security_check(user_input)
    if check_result:
        response = model(user_input)
        return response
    else:
        return "提问包含不合规信息"
```
</td>
</tr>
<tr>
<td>
C.
</td>
<td>

```python
def process_input(user_input):
    response = model(user_input)
    return response
```
</td>
</tr>
<tr>
<td>
D.
</td>
<td>

```python
def process_input(user_input):
    check_result = security_check(user_input)
    return check_result
```
</td>
</tr>
</tbody>
</table>


答案：B

解释： 输入的内容安全合规检查应该在模型处理输入之前进行。选项B首先对用户输入 user_input 进行安全检查，只有通过检查才会将输入传递给模型 model 处理。其他选项要么在模型处理后进行检查 (A)，要么完全没有检查 (C)，要么只返回检查结果而不处理模型调用 (D)。

<br>

【多选题】2.6.2. 你正在构建一个系统，用于检测用户上传图片中的违规文字信息。以下哪些技术或方法是必要的？（ ）

A. OCR (光学字符识别)

B. 文本合规检测

C. 使用多模态embedding模型获取图片的向量

D. 使用通义万相模型进行检测

E. 音频转文本

F. PHash 图像指纹识别

答案：AB

解释： A. OCR 用于从图片中提取文本信息，B. 文本合规检测用于分析提取的文本是否违规。C. 多模态embedding模型 用于多模态检索，D. 通义万相 用于生成图片，F. PHash 用于版权检查，都与检测违规文字信息无关。E. 音频转文本用于音频处理，与图片无关。

## ✉️ 评价反馈
感谢你学习阿里云大模型ACP认证课程，如果你觉得课程有哪里写得好、哪里写得不好，期待你[通过问卷提交评价和反馈](https://survey.aliyun.com/apps/zhiliao/Mo5O9vuie)。

你的批评和鼓励都是我们前进的动力。